# ＊PYTORCHによるBERTの実装


- 環境:Ubuntu18.04
- GPU:Quadro RTX 8000
- ドライバー:NVIDIA-SMI 460.32.03, Driver Version: 460.32.03, CUDA Version: 11.2
- 事前学習済みモデル: https://nlp.ist.i.kyoto-u.ac.jp/?ku_bert_japanese
    - ./weights: pytorch_model.bin, config.json
    - ./vocab: vocab.txt
    - ./data: train.tsv, test.tsv, test.csv
- 形態素解析時にストップワードを利用する場合はbert.pyの25行目と731行目のコメントアウトを外すこと
- EarlyStoppingを利用する場合はhttps://github.com/Bjarten/early-stopping-pytorch からpytorchtools.pyをutilsにインストールし学習・検証のコメントアウトを外すこと

# ＊事前準備


In [ ]:
!conda create -n pytorch_bert python=3.7
!conda activate pytorch_bert
!conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cudatoolkit=11.1 -c pytorch -c conda-forge
!conda install jupyter
!pip install transformers==2.9.0
!pip install torchtext==0.9.0
!pip install tqdm
!pip install pyknp
!pip install attrdict
!pip install spacy
!pip install mojimoji

# ＊ライブラリのインストール

In [ ]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext
import glob
import os
import io
import string
import re
import sys
import random
import spacy
import mojimoji
import pickle
from torchtext.vocab import Vectors
from utils.bert import BertTokenizer, load_vocab
from utils.config import PKL_FILE, VOCAB_FILE, DATA_PATH

In [ ]:
from utils.bert import BertTokenizer

max_length=128
path_result="./result/"
path_weights="./weights/"

# ＊前処理

In [ ]:
def preprocessing_text(text):
    # 半角・全角の統一（半角から全角へ変換）
    text = mojimoji.han_to_zen(text) 
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    #どっちでも
    #text = re.sub(',', '', text)

    # 数字文字の一律「0」化
    #text = re.sub(r'[0-9 ０-９]+', '0', text)  # 数字

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        #if (p == "."):
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")
        return text

# ＊TorchtextでDatasetの作成

In [ ]:
#形態素解析
#def DataLoaders_and_TEXT_train_val(max_length):
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 単語分割用のTokenizerを用意
tokenizer_bert = BertTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False)

# 前処理と単語分割をまとめた関数を定義
# 単語分割の関数を渡すので、tokenizer_bertではなく、tokenizer_bert.tokenizeを渡す点に注意
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)  # tokenizer_bert
    return ret

# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
#max_length=128
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True, lower=False, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token="[PAD]",unk_token='[UNK]')

LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False, dtype = torch.float)

# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path=DATA_PATH, train='./train.tsv',
    test='./test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])
#return train_val_ds, test_ds, TEXT

# ＊Dataloaderの作成

In [ ]:
batch_size = 32  # BERTでは16、32あたりを使用する

#test_dl, TEXT, dataloaders_dict, dataloaders_dict_val, test_ds= DataLoaders_and_TEXT_batch_train_val(train_val_ds, test_ds, TEXT, batch_size)
"""DataLoaderとTEXTオブジェクトを取得する。 """
    
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))
#print(train_val_ds)

vocab_bert, ids_to_tokens_bert = load_vocab(vocab_file=VOCAB_FILE)
TEXT.build_vocab(train_val_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert    

train_dl = torchtext.legacy.data.Iterator(train_val_ds, batch_size=batch_size, train=True)
train_dl_val = torchtext.legacy.data.Iterator(train_ds, batch_size=batch_size, train=True)
val_dl = torchtext.legacy.data.Iterator(val_ds, batch_size=batch_size, train=False, sort=False)
test_dl = torchtext.legacy.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict_val = {"train": train_dl_val, "val": val_dl}
dataloaders_dict = {"train": train_dl}
#dataloaders_dict = {"train": train_dl, "val": val_dl, "test": test_dl}
#dataloaders_dict = {"train": train_dl, "val": val_dl}

#return train_dl, val_dl, test_dl, TEXT, dataloaders_dict, dataloaders_dict, test_ds
#return test_dl, TEXT, dataloaders_dict, dataloaders_dict_val, test_ds

# ＊動作確認

In [ ]:
# 動作確認 テストデータのデータセットで確認
batch = next(iter(test_dl))
print(batch)
print(batch.Text)
print(batch.Label)

In [ ]:
from utils.bert import BertTokenizer
from utils.config import PKL_FILE, VOCAB_FILE, DATA_PATH
tokenizer_bert = BertTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False)

# ミニバッチの1文目を確認してみる
text_minibatch_1 = (batch.Text[0][1]).numpy()

# IDを単語に戻す
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)

# ＊BERTモデルを構築

In [ ]:
from utils.bert import get_config, BertModel, set_learned_params

# モデル設定のJOSNファイルをオブジェクト変数として読み込みます
config = get_config(file_path="./weights/config.json")

# BERTモデルを作成します
net_bert = BertModel(config)

# BERTモデルに学習済みパラメータセットします
net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")


In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, net_bert):
        super(BertClassifier, self).__init__()

        # BERTモジュール
        self.bert = net_bert  # BERTモデル

        # headに予測を追加
        # 入力はBERTの出力特徴量の次元、回帰分類の出力は1つ
        self.cls = nn.Linear(in_features=768, out_features=1)
        
        # ドロップアウト率：Hugging faceの実装
        #classifier_dropout = (config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob)
        self.dropout = nn.Dropout(0.1)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        
        # 書籍の実装：入力文章の1単語目[CLS]の特徴量を使用して、分類
        #vec_0 = encoded_layers[:, 0, :]
        #vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        #out = self.cls(vec_0)
        
        # hugging faceの実装:output
        pooled_output = self.dropout(pooled_output)
        out = self.cls(pooled_output)
        
        # その他
        # mean pooling:[PAD]以外のトークンで隠れ状態の平均をとる
        #averaged_hidden_state = \
        #    (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
        #    / attention_mask.sum(1, keepdim=True)
        
        # max pooling
        #vec_0, _ = encoded_layers.max(dim=1)  # max pooling
        #out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out


In [ ]:
# モデル構築
net = BertClassifier(net_bert)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


# ＊BERTのファインチューニング

## パターン1：全てのパラメータを更新

In [ ]:
# まず全部を、勾配計算Trueにしてしまう
for name, param in net.named_parameters():
    param.requires_grad = True

# 最適化手法の設定

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.cls.parameters(), 'lr': 1e-3}
], betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.MSELoss()

## パターン2：BERTの最終層と識別器のパラメータを更新

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 最後のBertLayerモジュールを勾配計算ありに変更
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for name, param in net.cls.named_parameters():
    param.requires_grad = True

# 最適化手法の設定

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-3}
], betas=(0.9, 0.999))

# 損失関数の設定
criterion =  nn.MSELoss()

## パターン3：BERTの最終4層と識別器のパラメータを更新

In [ ]:
# まずは全部OFF
for param in classifier.parameters():
    param.requires_grad = False

# BERTの最終4層分をON
for param in classifier.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

for param in classifier.bert.encoder.layer[-2].parameters():
    param.requires_grad = True

for param in classifier.bert.encoder.layer[-3].parameters():
    param.requires_grad = True

for param in classifier.bert.encoder.layer[-4].parameters():
    param.requires_grad = True

# クラス分類のところもON
for param in classifier.linear.parameters():
    param.requires_grad = True

# 事前学習済の箇所は学習率小さめ、最後の全結合層は大きめにする。
optimizer = optim.Adam([
    {'params': classifier.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': classifier.bert.encoder.layer[-2].parameters(), 'lr': 5e-5},
    {'params': classifier.bert.encoder.layer[-3].parameters(), 'lr': 5e-5},
    {'params': classifier.bert.encoder.layer[-4].parameters(), 'lr': 5e-5},
    {'params': classifier.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion  = nn.MSELoss()

# ＊学習・検証

## ＊ 開発データでハイパーパラメータを決定

In [ ]:
# モデルを学習させる関数を作成
#from utils.pytorchtools import EarlyStopping
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    #エポック数,Acuraccy,Loss保存用
    Epochs=[]
    Loss_train=[]
    Loss_val=[]
    
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    #early_stopping = EarlyStopping(patience=10)
    
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size
    n=0
    
    #時間
    start =time.time()
    
    # epochのループ
    for epoch in range(num_epochs):
        """
        if early_stopping.early_stop:
            print("Early Stopping")
            break # 打ち切り
        # epochごとの訓練と検証のループ
        """
        
        for phase in ['train', 'val']:
            count=0
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに
                count=1
            
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # Bertに入力
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_flg=False)

                    #loss = torch.sqrt(criterion(outputs, labels)) # 損失を計算
                    #loss = criterion(outputs, labels)
                    loss = criterion(outputs.squeeze(), labels.squeeze())
                    

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        
                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 本イテレーションのRMSE:{}'.format(iteration, loss.item(), duration, loss.item()))
                            t_iter_start = time.time()
                            
                            #early_stopping(loss.item(), net)
                                       
                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    
            # epochごとのloss
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss))
            
            t_epoch_start = time.time()
            
            if count == 0:
                Loss_train.append(epoch_loss)
            elif count==1:
                Loss_val.append(epoch_loss)
                
        Epochs.append(epoch+1)
    t=time.time()
    print("Time:{:.4f}sec".format(t-start))
        
    return net, Epochs, Loss_train, Loss_val


In [ ]:
# 学習・検証を実行する。1epochに20分ほどかかります
num_epochs = 30
net_trained, Epochs, Loss_train, Loss_val,  = train_model(net, dataloaders_dict_val,
                          criterion, optimizer, num_epochs=num_epochs)


## ＊学習時のEpochsごとのLossを出力

In [ ]:
import matplotlib.pyplot as plt
#plt.axes().set_aspect("equal")
#Loss_test=[]
plt.plot(Epochs, Loss_train,color="blue",label="train")
plt.plot(Epochs, Loss_val,color="red",label="val")
#plt.plot(Epochs,Loss_test,color="red",label="test")
plt.xticks(Epochs)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.savefig(path_result+"Loss_val.png")

## ＊ 全データで学習

In [ ]:
# モデルを学習させる関数を作成
#from utils.pytorchtools import EarlyStopping
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    #エポック数,Acuraccy,Loss保存用
    Epochs=[]
    Loss_train=[]
    Loss_val=[]
    
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    #early_stopping = EarlyStopping(patience=10)
    
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size
    n=0
    
    #時間
    start =time.time()
    # epochのループ
    for epoch in range(num_epochs):
        """
        if early_stopping.early_stop:
            print("Early Stopping")
            break # 打ち切り
        # epochごとの訓練と検証のループ
        """
        
        #for phase in ['train', 'val']:
        for phase in ['train']:
            count=0
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに
                count=1
            
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BertForに入力
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_flg=False)

                    #loss = torch.sqrt(criterion(outputs, labels)) # 損失を計算
                    #loss = criterion(outputs, labels)
                    loss = criterion(outputs.squeeze(), labels.squeeze())
                    

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        
                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 本イテレーションのRMSE:{}'.format(iteration, loss.item(), duration, loss.item()))
                            t_iter_start = time.time()
                            
                            #early_stopping(loss.item(), net)
                                       

                    iteration += 1

                    # 損失の合計を更新
                    epoch_loss += loss.item() * batch_size
                    
            # epochごとのloss
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss))
                
            t_epoch_start = time.time()
            
            if count == 0:
                Loss_train.append(epoch_loss)
            elif count==1:
                Loss_val.append(epoch_loss)
        Epochs.append(epoch+1)
    t=time.time()
    time_val = t-start
    print("Time:{:.4f}sec".format(time_val))
    
    return net, Epochs, Loss_train, Loss_val, time_val


In [ ]:
# 学習・検証を実行する。
num_epochs =10
net_trained, Epochs, Loss_train, Loss_val, time_val= train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

In [ ]:
# 学習したネットワークパラメータを保存します
save_path = path_weights+'bert_fine_tuning.pth'
torch.save(net_trained.state_dict(), save_path)

## ＊＊＊保存しているモデルを使用する場合＊＊＊
- BERTモデルを構築


In [ ]:
from utils.bert import get_config, BertModel, set_learned_params

# モデル設定のJOSNファイルをオブジェクト変数として読み込みます
config = get_config(file_path="./weights/config.json")

# BERTモデルを作成します
net_bert = BertModel(config)

# BERTモデルに学習済みパラメータセットします
net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")


In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, net_bert):
        super(BertClassifier, self).__init__()

        # BERTモジュール
        self.bert = net_bert  # BERTモデル

        # headに予測を追加
        # 入力はBERTの出力特徴量の次元、回帰分類の出力は1つ
        self.cls = nn.Linear(in_features=768, out_features=1)
        
        # ドロップアウト率：Hugging faceの実装
        #classifier_dropout = (config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob)
        self.dropout = nn.Dropout(0.1)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        
        # 書籍の実装：入力文章の1単語目[CLS]の特徴量を使用して、分類
        #vec_0 = encoded_layers[:, 0, :]
        #vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        #out = self.cls(vec_0)
        
        # hugging faceの実装:output
        pooled_output = self.dropout(pooled_output)
        out = self.cls(pooled_output)
        
        # その他
        # mean pooling:[PAD]以外のトークンで隠れ状態の平均をとる
        #averaged_hidden_state = \
        #    (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
        #    / attention_mask.sum(1, keepdim=True)
        
        # max pooling
        #vec_0, _ = encoded_layers.max(dim=1)  # max pooling
        #out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out

In [ ]:
# モデル構築
net_trained = BertClassifier(net_bert)

# 訓練モードに設定
net_trained.train()

print('ネットワーク設定完了')

#学習済みモデルを読み込む
model_path = './weights/bert_fine_tuning_dokokara_5.pth'
net_trained.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# 損失関数の設定
criterion =  nn.MSELoss()

## ＊＊＊Juman前処理用（テストデータのみ）＊＊＊

In [ ]:
from utils.dataloader import  DataLoaders_and_TEXT_test
max_length=128
batch_size=16
test_dl,TEXT, dataloaders_dict, test_ds=  DataLoaders_and_TEXT_test(max_length, batch_size)

#テストデータのみ
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
# 単語分割用のTokenizerを用意
tokenizer_bert = BertTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False)

def preprocessing_text(text):
    # 半角・全角の統一
    text = mojimoji.han_to_zen(text) 
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]+', '0', text)  # 数字

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        #if (p == "."):
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")
        return text

# 前処理と単語分割をまとめた関数を定義
# 単語分割の関数を渡すので、tokenizer_bertではなく、tokenizer_bert.tokenizeを渡す点に注意
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)  # tokenizer_bert
    return ret
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
#max_length = 256
#max_length=128
TEXT = torchtext.data.legacy.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True, lower=False, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token="[PAD]",unk_token='[UNK]')

LABEL = torchtext.data.legacy.Field(sequential=False, use_vocab=False, dtype = torch.float)

# フォルダ「data」から各tsvファイルを読み込みます
train_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path=DATA_PATH, train='train_dumy.tsv',
    test='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

test_dl = torchtext.legacy.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)
# 辞書オブジェクトにまとめる
dataloaders_dict = {"test": test_dl}

## ＊検証

In [ ]:
import pandas as pd
# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

predicted_label=[]#予測ラベル
ture_label = []
count=0

start =time.time()
for batch in tqdm(test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル
    epoch_loss=0.0

    
    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                              output_all_encoded_layers=False, attention_show_flg=False)

        #loss = torch.sqrt(criterion(outputs, labels)) # 損失を計算
        #loss = criterion(outputs, labels)
        loss = criterion(outputs.squeeze(), labels.squeeze())
        
        for i in range(batch_size):
            try:
                p_label = outputs[i].item()
                t_label = labels[i].item()
                
                predicted_label.append(p_label)
                ture_label.append(t_label)
                count+=1
            except:
                break
        
        
        epoch_loss += loss.item() * batch_size
        
epoch_loss = epoch_loss / len(test_dl.dataset)

t=time.time()
time_test =t-start
print("Time:{:.4f}sec".format(time_test))

print('テストデータ{}個でのRMSE：{:.4f}'.format(len(test_dl.dataset), epoch_loss))

df = pd.read_csv("./data/test.csv", names=("TEXT", "LABEL"), engine="python", encoding="utf-8-sig")
df["PREDICT"] = np.nan   #予測列を追加

for index in range(count):
    df.at[index, "PREDICT"] = predicted_label[index]
    
df.to_csv(path_result+"predicted_test.csv", encoding="utf-8-sig", index=False)


## ＊MSE・RMSEの出力

In [ ]:
from sklearn.metrics import mean_squared_error
mse_loss = mean_squared_error(ture_label, predicted_label, squared=True)
print("MSE：{}".format(mse_loss))

#import numpy as np
#rmse = np.sqrt(mean_squared_error(ture_label, predicted_label, squared=True))
rmse = mean_squared_error(ture_label, predicted_label, squared=False)
print("RMSE：{}".format(rmse))

## ＊MAEの出力

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(ture_label, predicted_label)
print("MAE{}".format(mae))

## ＊R2（決定係数）

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(ture_label, predicted_label)
print(r2)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# yyplot 作成関数
def yyplot(y_label, y_pred):
    yvalues = np.concatenate([y_label, y_pred])
    ymin, ymax, yrange = np.amin(yvalues), np.amax(yvalues), np.ptp(yvalues)
    plt.axes().set_aspect("equal")
    #plt.figure(figsize=(8, 8))
    plt.scatter(y_label, y_pred)
    plt.plot([ymin - yrange * 0.01, ymax + yrange * 0.01], [ymin - yrange * 0.01, ymax + yrange * 0.01])
    plt.xlim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.ylim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.xlabel('Label')
    plt.ylabel('Pred')
    plt.savefig(path_result+"pred.png")
    #plt.title('Observed-Predicted Plot')
    #plt.tick_params(labelsize=1)
    plt.show()
    return plt

# yyplot の実行例
np.random.seed(0)
y_obs = np.random.normal(size=(1000, 1))
y_pred = y_obs + np.random.normal(scale=0.3, size=(1000, 1))
fig = yyplot(ture_label, predicted_label)

In [ ]:
with open("{}RMSE.txt".format(path_result),"a",encoding="utf-8") as f:
    f.write("MSE：{}\n".format(mse_loss))
    f.write("RMSE：{}\n".format(rmse))
    f.write("MAE:{}\n".format(mae))
    f.write("R2:{}\n".format(r2))
    f.write("Time_val:{:.4f}sec\n".format(time_val))
    f.write("Time_test:{:.4f}sec\n".format(time_test))
    f.close()

# ＊Attentionの可視化

In [ ]:
# BertForIMDbで処理

# ミニバッチの用意
batch = next(iter(test_dl))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                       output_all_encoded_layers=False, attention_show_flg=True)

preds = outputs


In [ ]:
# HTMLを作成する関数を実装

from utils.bert import BertTokenizer
from utils.config import PKL_FILE, VOCAB_FILE, DATA_PATH
tokenizer_bert = BertTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False)
from utils.tokenizer import JumanTokenize

def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測
    
    label_str =  label 
    pred_str = pred.item()
    #print(pred)

    # 表示用のHTMLを作成する
    html = '正解スコア：{:.1f}<br>推論スコア：{}<br>'.format(label_str, pred_str)

    # Self-Attentionの重みを可視化。Multi-Headが12個なので、12種類のアテンションが存在
    for i in range(12):

        # indexのAttentionを抽出と規格化
        # 0単語目[CLS]の、i番目のMulti-Head Attentionを取り出す
        # indexはミニバッチの何個目のデータかをしめす
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()
        """
        html += '[BERTのAttentionを可視化_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 単語が[SEP]の場合は文章が終わりなのでbreak
            if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
            html += highlight(tokenizer_bert.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"
        """

    # 12種類のAttentionの平均を求める。最大値で規格化
    all_attens = attens*0  # all_attensという変数を作成する
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    #html += '[BERTのAttentionを可視化_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 単語が[SEP]の場合は文章が終わりなのでbreak
        
        if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break
        

        # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
        html += highlight(tokenizer_bert.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
        
    html += "<br><br>"

    return html


In [ ]:
from IPython.display import HTML

index = 5  # 出力させたいデータ
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


In [ ]:
count=0
for batch in test_dl:
    # ミニバッチの用意
    #batch = next(iter(test_dl))

    # GPUが使えるならGPUにデータを送る
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                           output_all_encoded_layers=False, attention_show_flg=True)
    preds = outputs
    
    for index in range(batch_size):
        try:
            html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML作成
            with open("{}Attention.html".format(path_result),"a", encoding="cp932") as f:
                f.write(html_output)
            count+=1
        except:
            pass 
#print(count)   
#HTML(html_output)  # HTML形式で出力